Usual magic stuff

In [1]:
import pythran

In [2]:
%load_ext pythran.magic

Create a pythran capsule, not callable as a Python function

In [3]:
%%pythran
#pythran export capsule f(int32, float64*, float64* )
def f(n, x, cp):
    c = cp[0]
    return c + x[0] - x[1] * x[2]

Prepare to pass this function to scipy's LowLevel

In [4]:
import ctypes
c = ctypes.c_double(1.0)
user_data = ctypes.cast(ctypes.pointer(c), ctypes.c_void_p)

In [5]:
from scipy import integrate, LowLevelCallable
func = LowLevelCallable(f, user_data, signature="double (int, double *, void *)")
dat =  [[0, 10], [-10, 0], [-1, 1]]
integrate.nquad(func, dat)

(1200.0000000000002, 1.3322676295501882e-11)

Showcase Numpy integration

In [6]:
%%pythran

# using numpy adaptator
#pythran export capsule transform(int64*, float64*, int32, int32, float64*)
from numpy.ctypeslib import as_array
def transform(output_coordinates, input_coordinates, output_rank, input_rank, user_data):
    shift = user_data[0]
    input_data = as_array(input_coordinates, input_rank)
    output_data = as_array(output_coordinates, output_rank)
    input_data[:] = output_data - shift
    return 1

# same with explicit loops
#pythran export capsule transform_basic(int64*, float64*, int32, int32, float64*)
def transform_basic(output_coordinates, input_coordinates, output_rank, input_rank, user_data):
    shift = user_data[0]
    for i in range(input_rank):
        input_coordinates[i] = output_coordinates[i] - shift;
    return 1

Validate output using the Numpy API

In [7]:
import ctypes
import numpy as np
from scipy import ndimage, LowLevelCallable


shift = 0.5

user_data = ctypes.c_double(shift)
ptr = ctypes.cast(ctypes.pointer(user_data), ctypes.c_void_p)
callback = LowLevelCallable(transform, ptr, "int (npy_intp *, double *, int, int, void *)")
im = np.arange(12).reshape(4, 3).astype(np.float64)

out0 = ndimage.geometric_transform(im, callback)
out0

array([[0.    , 0.    , 0.    ],
       [0.    , 1.3625, 2.7375],
       [0.    , 4.8125, 6.1875],
       [0.    , 8.2625, 9.6375]])

And using the explicit looping. Hopefully the results are the same!

In [8]:
callback = LowLevelCallable(transform_basic, ptr, "int (npy_intp *, double *, int, int, void *)")
im = np.arange(12).reshape(4, 3).astype(np.float64)

out1 = ndimage.geometric_transform(im, callback)
out1

array([[0.    , 0.    , 0.    ],
       [0.    , 1.3625, 2.7375],
       [0.    , 4.8125, 6.1875],
       [0.    , 8.2625, 9.6375]])

In [9]:
assert np.all(out0 == out1)